In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import statsmodels
import folium
import math

# Custom imports
from ipywidgets import IntProgress
from IPython.display import display
import time
from multiprocessing import Pool, Lock
import os
import json
import seaborn as sns
import time

**Question 3**

Is it possible to rank some patterns of land usage with social and environmental factors such as employment, life satisfaction (if data are presents), income inequalities (if data are presents), emissions and finally soil quality and sustainability?

In [3]:
dataLands = pd.read_csv("./data/fao_data_land_data.csv")
dataLands

country_or_area                                       element_code  \
0          Afghanistan                                                 11   
1          Afghanistan                                                 11   
2          Afghanistan                                                 11   
3          Afghanistan                                                 11   
4          Afghanistan                                                 11   
...                ...                                                ...   
104779              A   May include official, semi-official or estimat...   
104780              F                                        FAO Estimate   
104781              Q   Official data reported on FAO Questionnaires f...   
104782              Fm                                  Manual Estimation   
104783              I   Country data reported by International Organiz...   

       element    year     unit    value value_footnotes  \
0         Area  2007.0  1000 Ha  38661.0              Q    
1         Area  2006.0  1000 Ha  38659.0              Fm   
2         Area  2005.0  1000 Ha  38515.0              Fm   
3         Area  2004.0  1000 Ha  38064.0              Fm   
4         Area  2003.0  1000 Ha  38899.0              Fm   
...        ...     ...      ...      ...             ...   
104779     NaN     NaN      NaN      NaN             NaN   
104780     NaN     NaN      NaN      NaN             NaN   
104781     NaN     NaN      NaN      NaN             NaN   
104782     NaN     NaN      NaN      NaN             NaN   
104783     NaN     NaN      NaN      NaN             NaN   

                                  category  
0                        agricultural_area  
1                        agricultural_area  
2                        agricultural_area  
3                        agricultural_area  
4                        agricultural_area  
...                                    ...  
104779  total_area_equipped_for_irrigation  
104780  total_area_equipped_for_irrigation  
104781  total_area_equipped_for_irrigation  
104782  total_area_equipped_for_irrigation  
104783  total_area_equipped_for_irrigation  

[104784 rows x 8 columns]

In [ ]:
# investigate NaN values
dataLands[np.isnan(dataLands['value'].values)]

We observe that the dataset is pretty clean. The only NaNs in 'value' column are actually the footnotes so we can drop them.

In [14]:
dataLands = dataLands.dropna(subset=["value"])
dataLands.tail(5)

country_or_area element_code element    year     unit  value  \
104771        Zimbabwe           11    Area  1965.0  1000 Ha   34.0   
104772        Zimbabwe           11    Area  1964.0  1000 Ha   32.0   
104773        Zimbabwe           11    Area  1963.0  1000 Ha   23.0   
104774        Zimbabwe           11    Area  1962.0  1000 Ha   23.0   
104775        Zimbabwe           11    Area  1961.0  1000 Ha   22.0   

       value_footnotes                            category  
104771              Q   total_area_equipped_for_irrigation  
104772              Q   total_area_equipped_for_irrigation  
104773              Q   total_area_equipped_for_irrigation  
104774              Q   total_area_equipped_for_irrigation  
104775              Q   total_area_equipped_for_irrigation

# New Data
In order to answer the question at the beginning we need some new data:

Life expectancy at birth for both sexes combined (years):
http://data.un.org/Data.aspx?d=PopDiv&f=variableID%3a68

Value added by industries at current prices (ISIC Rev. 3)
http://data.un.org/Data.aspx?d=SNA&f=group_code%3a201

Employment by sex and economic activity
https://www.ilo.org/shinyapps/bulkexplorer5/?lang=en&segment=indicator&id=EMP_TEMP_SEX_ECO_NB_A

Non fatal occupational injuries per 100'000 workers by economic activity
https://www.ilo.org/shinyapps/bulkexplorer32/?lang=en&segment=indicator&id=INJ_NFTL_ECO_RT_A